In [1]:
%load_ext autoreload
%autoreload 2

import os
import wget
import zipfile
import numpy as np
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
from utils import *
from collections import Counter
from tqdm import tqdm
import time

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# import the graphs from the saved files
G_brighkite_checkins = nx.read_gpickle(os.path.join('data', 'brightkite', 'brightkite_checkins_graph.gpickle'))
G_gowalla_checkins = nx.read_gpickle(os.path.join('data', 'gowalla', 'gowalla_checkins_graph.gpickle'))
G_foursquareEU_checkins = nx.read_gpickle(os.path.join('data', 'foursquare', 'foursquareEU_checkins_graph.gpickle'))
G_foursquareIT_checkins = nx.read_gpickle(os.path.join('data', 'foursquare', 'foursquareIT_checkins_graph.gpickle'))

G_brighkite_friends = nx.read_gpickle(os.path.join('data', 'brightkite', 'brightkite_friendships_graph.gpickle'))
G_gowalla_friends = nx.read_gpickle(os.path.join('data', 'gowalla', 'gowalla_friendships_graph.gpickle'))
G_foursquareEU_friends = nx.read_gpickle(os.path.join('data', 'foursquare', 'foursquareEU_friendships_graph.gpickle'))
G_foursquareIT_friends = nx.read_gpickle(os.path.join('data', 'foursquare', 'foursquareIT_friendships_graph.gpickle'))

# open the dataframe object
analysis_results = pd.read_pickle('analysis_results.pkl')
analysis_results

,Graph,Number of Nodes,Number of Edges,Average Degree,Average Clustering Coefficient,log N,Average Shortest Path Length,betweenness centrality
0,Brightkite Checkins Graph,7191,3663807,1018.997914,0.702854,8.880586,2.411011,0.00022
1,Gowalla Checkins Graph,10702,303104,56.644366,0.505597,9.278186,5.222903,0.000301
2,Foursquare EU Checkins Graph,20282,7430376,732.706439,0.597097,9.917489,2.2843,0.000089
3,Foursquare IT Checkins Graph,3730,629749,337.667024,0.683565,8.224164,2.185477,0.000428
4,Brightkite Friendship Graph,5928,34673,11.698043,0.219749,8.687442,5.052162,0.000448
5,(Filtered) Gowalla Friendship Graph,8396,29122,6.937113,0.217544,9.035511,4.558532,0.000357
6,Foursquare IT Friendship Graph,2073,6217,5.99807,0.148489,7.636752,19.530752,0.000879
7,Foursquare EU Friendship Graph,16491,59419,7.206234,0.167946,9.710570,23.713864,0.000272


The first thing that we want to do is very simple, create a random reference for each graph

In [3]:
# analysis_results = pd.DataFrame(columns=['Graph', 'Number of Nodes', 'Number of Edges', 'Average Degree', 'Average Clustering Coefficient', 'log N', 'Average Shortest Path Length', 'betweenness centrality'], index=None)

checkins_graphs = [G_brighkite_checkins, G_gowalla_checkins, G_foursquareEU_checkins, G_foursquareIT_checkins]
friendships_graph = [G_brighkite_friends, G_gowalla_friends, G_foursquareIT_friends, G_foursquareEU_friends]

graphs_all = checkins_graphs + friendships_graph

# Random shit

In [ ]:
# analysis_results_erods = pd.DataFrame(columns=['Graph', 'Number of Nodes', 'Number of Edges', 'Average Degree', 'Average Clustering Coefficient', 'log N', 'Average Shortest Path Length', 'betweenness centrality'], index=None)

# analysis_results_ws = pd.DataFrame(columns=['Graph', 'Number of Nodes', 'Number of Edges', 'Average Degree', 'Average Clustering Coefficient', 'log N', 'Average Shortest Path Length', 'betweenness centrality'], index=None)

# for graph in graphs_all:
#     print("\nCreating random graph for graph: ", graph.name)
#     G_erd = create_random_graphs(graph, model='erdos', save=False)
#     G_ws = create_random_graphs(graph, model='watts_strogatz', save=False)
    
#     # add the basic information to the dataframe
#     analysis_results_erods = analysis_results_erods.append({
#         'Graph': G_erd.name,
#         'Number of Nodes': G_erd.number_of_nodes(),
#         'Number of Edges': G_erd.number_of_edges(),
#         'log N': np.log(G_erd.number_of_nodes())
#     }, ignore_index=True)

#     # add the basic information to the dataframe
#     analysis_results_ws = analysis_results_ws.append({
#         'Graph': G_ws.name,
#         'Number of Nodes': G_ws.number_of_nodes(),
#         'Number of Edges': G_ws.number_of_edges(),
#         'log N': np.log(G_ws.number_of_nodes())
#     }, ignore_index=True)

#     # compute the average degree and add it to the dataframes
#     avg_deg_erd = np.mean([d for n, d in G_erd.degree()])
#     avg_deg_ws = np.mean([d for n, d in G_ws.degree()])
#     analysis_results_erods.loc[analysis_results_erods['Graph'] == G_erd.name, 'Average Degree'] = avg_deg_erd
#     analysis_results_ws.loc[analysis_results_ws['Graph'] == G_ws.name, 'Average Degree'] = avg_deg_ws

#     # compute the average clustering coefficient and add it to the dataframes
#     avg_clustering_erd = average_clustering_coefficient(G_erd, k = 0.9)
#     avg_clustering_ws = average_clustering_coefficient(G_ws, k = 0.9)
#     analysis_results_erods.loc[analysis_results_erods['Graph'] == G_erd.name, 'Average Clustering Coefficient'] = avg_clustering_erd
#     analysis_results_ws.loc[analysis_results_ws['Graph'] == G_ws.name, 'Average Clustering Coefficient'] = avg_clustering_ws

#     # compute the average shortest path length and add it to the dataframes
#     average_shortest_path_length_erd = average_shortest_path(G_erd, k = 0.9)
#     average_shortest_path_length_ws = average_shortest_path(G_ws, k = 0.9)
#     analysis_results_erods.loc[analysis_results_erods['Graph'] == G.name, 'Average Shortest Path Length'] = average_shortest_path_length_erd
#     analysis_results_ws.loc[analysis_results_ws['Graph'] == G.name, 'Average Shortest Path Length'] = average_shortest_path_length_ws

#     # compute the betweenness centrality and add it to the dataframes
#     betweenness_centrality_erd = np.mean(list(betweenness_centrality_parallel(G_erd, 4, k = 0.9).values()))
#     betweenness_centrality_ws = np.mean(list(betweenness_centrality_parallel(G_ws, 4, k = 0.9).values()))
#     analysis_results_erods.loc[analysis_results_erods['Graph'] == G.name, 'betweenness centrality'] = betweenness_centrality_erd
#     analysis_results_ws.loc[analysis_results_ws['Graph'] == G.name, 'betweenness centrality'] = betweenness_centrality_ws

#     # save memory
#     del G_erd, G_ws

# analysis_results_erods.to_pickle('analysis_results_erods.pkl')
# analysis_results_ws.to_pickle('analysis_results_ws.pkl')

# Small Worldness


We have already computed the average clusting coefficient and the average shortesh path len for our networks. We can save a lot of time by skipping this computations

In [4]:
def omega(G, C_og, L_og, niter, nrand):
    randMetrics = {"C": [], "L": []}

    # Calculate initial average clustering coefficient which potentially will
    # get replaced by higher clustering coefficients from generated lattice
    # reference graphs
    Cl = C_og

    niter_lattice_reference = niter
    niter_random_reference = niter * 2

    for _ in range(nrand):
        
        # Generate random graph
        Gr = nx.random_reference(G, niter=niter_random_reference, seed=42)
        randMetrics["L"].append(nx.average_shortest_path_length(Gr))

        # Generate lattice graph
        Gl = nx.lattice_reference(G, niter=niter_lattice_reference, seed=42)

        # Replace old clustering coefficient, if clustering is higher in
        # generated lattice reference
        Cl_temp = nx.average_clustering(Gl)
        if Cl_temp > Cl:
            Cl = Cl_temp

    C = C_og
    L = L_og
    Lr = np.mean(randMetrics["L"])

    omega = (Lr / L) - (C / Cl)

In [5]:
analysis_results = pd.read_pickle('analysis_results.pkl')

omega_results = pd.DataFrame(columns=['Graph', 'omega'])

for G in checkins_graphs:
    print(G.name)
    C_og = analysis_results.loc[analysis_results['Graph'] == G.name, 'Average Clustering Coefficient'].values[0]
    L_og = analysis_results.loc[analysis_results['Graph'] == G.name, 'Average Shortest Path Length'].values[0]

    omega = omega(G, C_og, L_og, 2, 3)
    
    omega_results = omega_results.append({
        'Graph': G.name,
        'omega': omega
    }, ignore_index=True)

Brightkite Checkins Graph


In [ ]:
for G in friendships_graphs:
    print(G.name)
    C_og = analysis_results.loc[analysis_results['Graph'] == G.name, 'Average Clustering Coefficient'].values[0]
    L_og = analysis_results.loc[analysis_results['Graph'] == G.name, 'Average Shortest Path Length'].values[0]

    omega = omega(G, C_og, L_og, 2, 3)
    
    omega_results = omega_results.append({
        'Graph': G.name,
        'omega': omega
    }, ignore_index=True)